# Projeto 1 - Ciência dos Dados

Nome: Mia Valentin Machado

Nome: Matheus Castellucci

Nome: Pedro Henrique

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [12]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
#import emoji

In [13]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Matheus Castellucci\Desktop\Insper\Semestre 2\CieDados\Projeto 1\Projeto-1-Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [14]:
PS5 = 'PS5 - Final.xlsx'

In [15]:
train = pd.read_excel(PS5)
train.head(5)

,Tweet,Classificação de Relevância
0,um ps5 eh pra sempre https://t.co/ow1aobrsmm,2
1,"vi uma galerinha falando: ""ain que maravilha w...",0
2,"@jisegirl os dois, você no sofa, o ps5 no rack...",0
3,vender meu pulmão pra comprar um ps5 https://t...,2
4,@midiasonysta @playstation_br endosso o coro.....,1


In [16]:
test = pd.read_excel(PS5, sheet_name = 'Teste')
test.head(5)

,Tweet,Classificação de Relevância
0,"@wagnerthomazoni diria que é bom até, mais tem...",2
1,@d_ultra_ @esportzy @vplportugalps eu já que s...,1
2,@guisouzadias branco com ps5 em casa adora def...,0
3,@oblivionhell jogando no meu ps5,2
4,"queria pegar um console mais barato, mas exclu...",3


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O nosso produto é o PS5, um console mais recente da Sony, uma empresa predominante no mercado de eletrônicos. Diante disto, consideramos como relevantes tweets com teor critico a cerca do console e suas particuliaridades, classificando então as opniões sobre seus jogos e manisfestações sobre o desejo de possuir um PS5, como tweets irrelevantes ou neutros dependendo do teor do tweet que os um possível comprador 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**